In [9]:
import random

from typing import List, Tuple, Optional

import os
import sys
from pathlib import Path

sys.path.append(str((Path(os.path.abspath("")) / "..").resolve()))
from sudoku_variants import SudokuAI
from sudoku_variants.const import DIGITS, NUM_COL, NUM_ROW
from sudoku_variants.rule import Rule, WithData
from sudoku_variants.func import rules as R

In [10]:
# We will implement a stricter version of Consecutive and call it ConsecutiveStrict
# In this version, unmarked adjacent cells must be non-consecutive

ConsecutiveDataType = List[Tuple[Tuple[int, int], Tuple[int, int]]]

# first bool is to disable rule check when initially the data is not populated yet
ConsecutiveStrictDataType = Tuple[bool, ConsecutiveDataType]


class ConsecutiveStrict(Rule, WithData):
    offsets: List[Tuple[int, int]] = [
        (-1, 0),
        (1, 0),
        (0, -1),
        (0, 1),
    ]

    def __init__(self, data: Optional[ConsecutiveStrictDataType] = None) -> None:
        """
        data: a flag to activate the rule check and a list of neighbours which is a pair of coordinates
        """
        super().__init__()
        self.data: ConsecutiveStrictDataType
        if data is None:
            self.data = (False, [])
        else:
            self.data = data

    def check_move(self, board: List[List[int]], row: int, col: int, digit: int) -> bool:
        if digit not in DIGITS:
            return True

        # disable rule check
        if not self.data[0]:
            return True

        cur_coord = (row, col)
        for offset in self.offsets:
            neighbour_coord = (row + offset[0], col + offset[1])
            pair1 = (cur_coord, neighbour_coord)
            pair2 = (neighbour_coord, cur_coord)

            if pair1 in self.data[1] or pair2 in self.data[1]:
                neighbour_digit = board[neighbour_coord[0]][neighbour_coord[1]]
                if neighbour_digit in DIGITS and abs(digit - neighbour_digit) != 1:
                    return False
            else:
                # unmarked adjacent cells must be non-consecutive
                if (0 <= neighbour_coord[0] < len(board)) and (0 <= neighbour_coord[1] < len(board[0])):
                    neighbour_digit = board[neighbour_coord[0]][neighbour_coord[1]]
                    if neighbour_digit in DIGITS and abs(digit - neighbour_digit) == 1:
                        return False

        return True

    def populate_initial_data(self):
        self.data = (False, [])

    def extract_data_from_board(self, board: List[List[int]]):
        consecutive_data: ConsecutiveDataType = []
        for i in range(NUM_ROW):
            for j in range(NUM_COL):
                cur_coord = (i, j)
                cur_digit = board[i][j]

                for offset in self.offsets:
                    if (0 <= i + offset[0] < len(board)) and (0 <= j + offset[1] < len(board[0])):
                        neighbour_coord = (i + offset[0], j + offset[1])
                        neighbour_digit = board[neighbour_coord[0]][neighbour_coord[1]]
                        if abs(cur_digit - neighbour_digit) == 1:
                            consecutive_data.append((cur_coord, neighbour_coord))

        size = random.randint(0, len(consecutive_data))
        consecutive_data = random.sample(consecutive_data, size)
        # activate the rule check
        self.data = (True, consecutive_data)

In [11]:
consecutive_strict_rule = ConsecutiveStrict()
rules = R.with_standard_rules([consecutive_strict_rule])
print(R.to_name(rules))

ConsecutiveStrict, Orthogonal, SubBoard


In [14]:
sudoku = SudokuAI.generate(rules, max_erased=5)
print(sudoku)
print(consecutive_strict_rule.data)

Rules: ConsecutiveStrict, Orthogonal, SubBoard
+-------+-------+-------+
| 8 7 4 | 9 3 1 | 2 * 5 |
| 6 2 3 | 4 8 5 | 7 1 9 |
| 1 9 5 | 2 6 * | 8 3 4 |
+-------+-------+-------+
| 4 5 6 | 7 1 * | 9 8 3 |
| 9 3 1 | 5 4 8 | 6 7 * |
| 2 8 7 | 3 9 * | 4 5 1 |
+-------+-------+-------+
| 3 4 8 | 1 7 9 | 5 2 6 |
| 5 * 9 | * * 3 | 1 4 7 |
| 7 * 2 | 6 5 * | 3 9 8 |
+-------+-------+-------+
(True, [((0, 7), (0, 8)), ((1, 6), (2, 6)), ((3, 2), (3, 1)), ((8, 2), (8, 1)), ((7, 1), (7, 0)), ((0, 8), (0, 7)), ((3, 1), (3, 2)), ((5, 0), (6, 0)), ((3, 8), (2, 8)), ((5, 6), (5, 7)), ((0, 1), (0, 0)), ((3, 7), (3, 6)), ((1, 2), (1, 1)), ((8, 8), (8, 7)), ((1, 1), (1, 2)), ((7, 2), (7, 3)), ((3, 0), (3, 1)), ((2, 8), (2, 7)), ((8, 5), (8, 4)), ((6, 1), (6, 0)), ((5, 2), (6, 2)), ((3, 2), (2, 2)), ((2, 6), (1, 6)), ((8, 5), (8, 6)), ((0, 0), (0, 1)), ((3, 6), (2, 6)), ((8, 3), (8, 4)), ((3, 5), (3, 4)), ((3, 3), (3, 2)), ((2, 6), (2, 5)), ((4, 7), (4, 6)), ((2, 6), (3, 6)), ((8, 1), (8, 2)), ((6, 6), (5, 